# Tomato Leaf Disease Detection 0.998 [inference]

### Hi kagglers, This is `inference` notebook using `Keras`.

> 
>  [Tomato Leaf Disease Detection 0.998 [Training]](https://www.kaggle.com/ammarnassanalhajali/tomato-leaf-disease-detection-0-998-training)



### Please if this kernel is useful, <font color='red'>please upvote !!</font>

In [ ]:
import os, cv2, json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam

from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D

In [ ]:
train= pd.read_csv("../input/tomato-diseases-dataset-csvimages/train.csv")

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_validate, y_train, y_test = train_test_split(train, train.label, 
                                                    train_size=0.8, 
                                                    random_state=42,
                                                    stratify=train.label)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_validate = df_validate.reset_index(drop=True)

In [ ]:
sample = df_train[df_train.label == 3].sample(3)
plt.figure(figsize=(15, 5))
for ind, (img, label) in enumerate(zip(sample.img, sample.label)):
    plt.subplot(1, 3, ind + 1)
    img = cv2.imread(os.path.join("../input/tomato-diseases-dataset-csvimages/Tomato_images/Tomato_images", img))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis("off")
    
plt.show()

In [ ]:
# Main parameters
BATCH_SIZE = 16
STEPS_PER_EPOCH = len(train)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train)*0.2 / BATCH_SIZE
EPOCHS =60 #
IMG_WIDTH= 256
IMG_HEIGHT= 256
train_dir = "../input/tomato-diseases-dataset-csvimages/Tomato_images/Tomato_images"

In [ ]:
df_train.label = df_train.label.astype('str')
df_validate.label = df_validate.label.astype('str')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               rotation_range = 180,
                               vertical_flip = True,
                               horizontal_flip = True)
# our train_datagen generator will use the following transformations on the images
validation_datagen = ImageDataGenerator(rescale=1./255)





train_generator = train_datagen.flow_from_dataframe(df_train, 
                                                    train_dir,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    x_col='img',
                                                    y_col='label',
                                                    class_mode = 'categorical')

# generator = ImageDataGenerator(*args).flow_from_dataframe(dataframe, directory, target_size,
# batch_size, x_col, y_col, class_mode)
# your dataframe shoudl be in the format such that x_col = features, y_col = class/label
# binary class mode since output is either 0(dog) or 1(cat)

validation_generator = validation_datagen.flow_from_dataframe(df_validate, 
                                                   train_dir,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    x_col='img',
                                                    y_col='label',
                                                    class_mode='categorical', 
                                                  batch_size=BATCH_SIZE)

In [ ]:
def create_model():
    efficientnet_layers = InceptionV3(weights='imagenet', 
                                         include_top=False, 
                                         input_shape = (IMG_WIDTH, IMG_HEIGHT, 3),
                                         pooling='avg')

    model = Sequential()
    model.add(efficientnet_layers)
    model.add(Dense(10, activation="softmax"))
    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "categorical_crossentropy",
                  metrics = ["acc"])

    return model


In [ ]:
model = create_model()
model.summary()

In [ ]:
model.load_weights('../input/tomatoleafdiseasedetection-weights/InceptionV3_256.h5')

In [ ]:
#ss=df_validate.sample(n=20)
ss=df_validate
ss=ss[['img', 'label']]

preds = []

for image_id in ss.img:
    image = Image.open(os.path.join("../input/tomato-diseases-dataset-csvimages/Tomato_images/Tomato_images/", image_id))
    array = tf.keras.preprocessing.image.img_to_array(image)
    array=array/255
    image = np.expand_dims(array, axis = 0)
    preds.append(np.argmax(model.predict(image)))

ss['labelP'] = preds
ss


In [ ]:
score = model.evaluate_generator(validation_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
confusion_matrix = pd.crosstab(ss.label, ss.labelP, rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

In [ ]:

plt.figure(figsize=(10,8))

#use seaborn to draw the headmap
sns.heatmap(confusion_matrix, 
            xticklabels=confusion_matrix.columns.values, #x label
            yticklabels=confusion_matrix.columns.values,cmap="YlGnBu" ,annot=True, fmt="d")
plt.show()

In [ ]:
from imblearn.metrics import sensitivity_score, specificity_score


from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score, confusion_matrix
y_test=ss.label.values.astype(int)
y_pred=ss.labelP.values.astype(int)

type(y_test)
# Print f1, precision, and recall scores
print("specificity:",specificity_score(y_test, y_pred , average="macro"))
print("sensitivity:",sensitivity_score(y_test, y_pred , average="macro"))
print("recall:",recall_score(y_test, y_pred , average="macro"))
print("precision::",precision_score(y_test, y_pred , average="macro"))
print("f1_score:",f1_score(y_test, y_pred , average="macro"))
print("accuracy_score:",accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
import numpy as np


print(classification_report(y_test, y_pred))

In [ ]:
y_true = y_test
y_prediction = y_pred
cnf_matrix = confusion_matrix(y_true, y_prediction)
print(cnf_matrix)
#[[1 1 3]
# [3 2 2]
# [1 3 1]]

FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print("Sensitivity OR recall")
print(TPR)
print("-------------------")
print("Specificity")
print(TNR)
print("-------------------")
print("Precision")
print(PPV)
print("-------------------")
print("accuracy")
print(ACC)
